In [1]:
import sys
sys.path.append('..')
import logging
from lib.utils import read_config
# Google API
from lib.googlesheet.authen import login as ggsheet_login
from lib.googlesheet import utils as ggsheet_utils
import pandas as pd
import random
import numpy as np
from scipy.stats import norm
from tqdm import tqdm
import datetime

In [7]:
config = read_config('config/cdo.yaml')
# get data google sheet
service = ggsheet_login(config['service_account_file'])
loan_data = ggsheet_utils.get_data(service=service, spreadsheet_id=config['google_sheet']['id'], sheet_name='Data Input')

def convert_pct_to_float(arr):
    return arr.str.replace('%','').astype(float)/100

def gen_random():
    while(True):
        nrnd = 2*random.random() - 1
        nrnd2 = 2*random.random() - 1
        w = nrnd**2 + nrnd2**2
        if(w<1):
            break
    return (-2*np.log(w)/w)**0.5*nrnd

df_loan = pd.DataFrame(loan_data[1:], columns = loan_data[0])
df_loan['PD'] = convert_pct_to_float(df_loan['PD'])
df_loan['LGD'] = convert_pct_to_float(df_loan['LGD'])
df_loan['FactorSensitivity'] = convert_pct_to_float(df_loan['FactorSensitivity'])
df_loan['EAD'] = df_loan['EAD'].astype(float)
df_loan['pd_inv'] = norm.ppf(df_loan['PD'])
sumEAD = df_loan['EAD'].sum()
df_loan['EAD_pct'] = df_loan['EAD']/sumEAD
df_loan['w2'] = (1-df_loan['FactorSensitivity']**2)**0.5

tranche_config = ggsheet_utils.get_data(service=service, spreadsheet_id=config['google_sheet']['id'], sheet_name='Tranche Structure')

df_tranche = pd.DataFrame(tranche_config[1:], columns = tranche_config[0])
df_tranche['Attachment'] = convert_pct_to_float(df_tranche['Attachment'])
df_tranche['Detachment'] = convert_pct_to_float(df_tranche['Detachment'])

num_simulation = int(ggsheet_utils.get_data(service=service, spreadsheet_id=config['google_sheet']['id'], sheet_name='Risk Analysis')[0][1])

df_tranche['PD'] = 0
df_tranche['EL'] = 0
df_tranche['updated_time'] = datetime.datetime.now().strftime('%y/%m/%d %H:%M')
# Conduct M Monte Carlo trials
for trial in tqdm(range(num_simulation)):
    factor = gen_random()
    # Compute portfolio loss for one trial
    loss_single = 0
    for loan in df_loan.iterrows():
        if(loan[1]['FactorSensitivity']*factor + loan[1]['w2']*gen_random() < loan[1]['pd_inv']):
            loss_single += loan[1]['LGD']*loan[1]['EAD_pct']

    # Record losses for tranches
    for tran in range(len(df_tranche)):
        if(loss_single > df_tranche.loc[tran, 'Attachment']):
            df_tranche.loc[tran, 'PD'] = df_tranche.loc[tran, 'PD'] + 1/num_simulation
            val = (loss_single - df_tranche.loc[tran, 'Attachment'])/(df_tranche.loc[tran, 'Detachment'] - df_tranche.loc[tran, 'Attachment'])
            df_tranche.loc[tran, 'EL'] = df_tranche.loc[tran, 'EL'] + np.min([val, 1])/num_simulation

ggsheet_utils.delete_data(service=service, spreadsheet_id=config['google_sheet']['id'], sheet_name='Risk Analysis', position_delete=f'A4:Z')
ggsheet_utils.update_data(service=service, spreadsheet_id=config['google_sheet']['id'], sheet_name='Risk Analysis', data=df_tranche.values.tolist(), position_update=f'A4')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:38<00:00, 259.00it/s]
